In [ ]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import KFold
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint, Callback
import wandb
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # This line prevents TensorFlow from using GPU

# Initialize wandb
wandb.init(project="computer_vision", config={
    "learning_rate": 0.0005,
    "architecture": "DenseNet121",
    "dataset": "alzheimer",
    "epochs": 40,
    "batch_size": 16,
    "input_shape": (176, 208, 3),
    "num_classes": 4
})
config = wandb.config

class CustomWandbCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        wandb.log({
            'epoch': epoch,
            'loss': logs.get('loss', 0),
            'accuracy': logs.get('accuracy', 0),
            'val_loss': logs.get('val_loss', 0),
            'val_accuracy': logs.get('val_accuracy', 0)
        })

# Function to load all images and labels
def load_images_and_labels(dataset_path, target_size):
    datagen = ImageDataGenerator(rescale=1./255)
    generator = datagen.flow_from_directory(
        dataset_path,
        target_size=target_size[:2],
        batch_size=config.batch_size,
        class_mode='categorical',
        shuffle=False  # Important for maintaining label order for k-fold
    )
    images, labels = [], []
    for _ in range(len(generator)):
        img, lbl = next(generator)
        images.append(img)
        labels.append(lbl)
    images = np.vstack(images)
    labels = np.vstack(labels)
    return images, labels

# Load training and test data
train_images, train_labels = load_images_and_labels('train', config.input_shape)
test_images, test_labels = load_images_and_labels('test', config.input_shape)

def create_densenet_model():
    base_model = DenseNet121(weights='imagenet', include_top=False, input_tensor=tf.keras.Input(shape=config.input_shape))
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(config.num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=config.learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

fold_var = 1
for train_index, val_index in kf.split(train_images):
    x_train, x_val = train_images[train_index], train_images[val_index]
    y_train, y_val = train_labels[train_index], train_labels[val_index]

    model = create_densenet_model()

    # Train model
    checkpoint_path = f"model_checkpoints/model_fold_{fold_var}.keras"
    checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=config.epochs,
        batch_size=config.batch_size,
        callbacks=[checkpoint, CustomWandbCallback()]
    )

    # Evaluate on the test set after each fold
    test_loss, test_accuracy = model.evaluate(test_images, test_labels)
    print(f"Test Results - Loss: {test_loss}, Accuracy: {test_accuracy}")
    wandb.log({'test_loss_fold_'+str(fold_var): test_loss, 'test_accuracy_fold_'+str(fold_var): test_accuracy})

    # Plot training and validation accuracy and loss
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'Fold {fold_var} Training vs Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'Fold {fold_var} Training vs Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    fold_var += 1

# Close the WandB run
wandb.finish()


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Load the models
model_1 = tf.keras.models.load_model('model_checkpoints/model_fold_1.keras')
model_2 = tf.keras.models.load_model('model_checkpoints/model_fold_2.keras')
model_3 = tf.keras.models.load_model('model_checkpoints/model_fold_3.keras')
model_4 = tf.keras.models.load_model('model_checkpoints/model_fold_4.keras')
model_5 = tf.keras.models.load_model('model_checkpoints/model_fold_5.keras')

# Assuming test_datagen and test_generator are already defined as shown previously
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    'test',  # Update this path to your test directory
    target_size=(176, 208),  # Ensure these are the dimensions expected by your models
    batch_size=16,           # Match this with the batch size used during training
    class_mode='categorical',
    shuffle=False)           # Important to set shuffle to False for evaluation purposes

# Predict the classes with the model
predictions_1 = model_1.predict(test_generator)
predictions_2 = model_2.predict(test_generator)
predictions_3 = model_3.predict(test_generator)
predictions_4 = model_4.predict(test_generator)
predictions_5 = model_5.predict(test_generator)

# Average the predictions for soft voting or choose other voting mechanism
average_predictions = (predictions_1 + predictions_2 + predictions_3 + predictions_4 + predictions_5 ) / 5
predicted_classes = np.argmax(average_predictions, axis=1)

# Since the generator might shuffle the order of the data, the classes array needs to correspond correctly
true_classes = test_generator.classes

# Evaluate the ensemble's accuracy manually if needed
accuracy = np.mean(predicted_classes == true_classes)
print(f"Ensemble Test Accuracy: {accuracy}")

# Generate the confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=test_generator.class_indices.keys())

# Display the confusion matrix
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix for Ensemble')
plt.show()


In [ ]:
# Import necessary libraries, and k-fold with gabor
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint
import wandb
from wandb.keras import WandbCallback

# Initialize WandB
wandb.init(project="computer_vision", config={
    "learning_rate": 0.0005,
    "epochs": 100,
    "batch_size": 16,
    "input_shape": (176, 208, 3),
    "num_classes": 4
})
config = wandb.config

def load_images_and_labels(base_path):
    images = []
    labels = []
    label_map = {}
    class_id = 0
    for folder_name in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder_name)
        if os.path.isdir(folder_path):
            for filename in os.listdir(folder_path):
                if filename.lower().endswith((".jpg", ".png")):
                    img_path = os.path.join(folder_path, filename)
                    img = load_img(img_path, target_size=(config.input_shape[0], config.input_shape[1]))
                    img = img_to_array(img)
                    img = preprocess_input(img)
                    images.append(img)
                    labels.append(class_id)
            label_map[folder_name] = class_id
            class_id += 1
    return np.array(images), np.array(labels), label_map

# Load training data
dataset_path = 'alzheimer_gabor_8/train'
images, labels, label_map = load_images_and_labels(dataset_path)
labels = to_categorical(labels, num_classes=config.num_classes)

def create_resnet_model():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=config.input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(config.num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(config.learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# K-Fold Cross-Validation
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold_var, (train_idx, val_idx) in enumerate(kfold.split(images, np.argmax(labels, axis=1)), 1):
    train_data, val_data = images[train_idx], images[val_idx]
    train_labels, val_labels = labels[train_idx], labels[val_idx]
    train_generator = ImageDataGenerator()
    val_generator = ImageDataGenerator()
    train_gen = train_generator.flow(train_data, train_labels, batch_size=config.batch_size)
    val_gen = val_generator.flow(val_data, val_labels, batch_size=config.batch_size)

    model = create_resnet_model()
    checkpoint_path = f"best_model_fold_{fold_var}_epoch{{epoch:02d}}_val_acc{{val_accuracy:.4f}}.keras"
    checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, verbose=1, mode='max')

    # Adding WandbCallback to log metrics after each epoch
    history = model.fit(train_gen, epochs=config.epochs, validation_data=val_gen, 
                        callbacks=[checkpoint])

    # Optionally log summary data at the end of each fold
    logs = {}
    for metric in history.history:
        logs[f'{metric}_fold_{fold_var}'] = history.history[metric][-1]
    wandb.log(logs)

# Load and preprocess test data
test_path = 'alzheimer_gabor_8/test'
test_images, test_labels, _ = load_images_and_labels(test_path)
test_labels = to_categorical(test_labels, num_classes=config.num_classes)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_gen = test_datagen.flow(test_images, test_labels, batch_size=config.batch_size)

final_checkpoint = 'final_best_model_epoch{epoch:02d}_val_acc{val_accuracy:.4f}.keras'
final_model = create_resnet_model()
final_checkpoint_callback = ModelCheckpoint(final_checkpoint, monitor='val_accuracy', save_best_only=True, verbose=1, mode='max')
final_model.fit(images, labels, epochs=config.epochs, batch_size=config.batch_size, validation_data=test_gen, callbacks=[final_checkpoint_callback])

best_final_model = tf.keras.models.load_model(final_checkpoint)
test_loss, test_accuracy = best_final_model.evaluate(test_gen)
print(f'Final Test loss: {test_loss}, Test accuracy: {test_accuracy}')
wandb.log({'final_test_loss': test_loss, 'final_test_accuracy': test_accuracy})

# Close the WandB run
wandb.finish()
